In [1]:
import json
import datetime, pytz
import speech_recognition as sr

from db_search import get_info

from langchain_mistralai import ChatMistralAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import ToolMessage, HumanMessage, SystemMessage, AIMessage
from langchain.tools import tool, StructuredTool

In [2]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    api_key="suFEUjmnfvT2AQSF5b6MQJ9NyZwDLsqG"
)

In [3]:
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("_"*100)
        try:
            audio = recognizer.listen(source, timeout=300)
            user_input = recognizer.recognize_google(audio)
            print(f"You said: {user_input}")
            return user_input
        except sr.UnknownValueError:
            return None
        except sr.WaitTimeoutError:
            return None

In [4]:
@tool

def get_user_data() -> dict:
    """
    Returns all information about the customer's loan.
    """
    key = int(phone)
    user_data = get_info(key)
    return user_data

In [5]:
@tool
def current_date_time() -> dict:
    '''
    Returns the current server date and time in JSON format.
    '''
    now = datetime.datetime.now()
    ist_timezone = pytz.timezone('Asia/Kolkata')
    dt_ist = now.astimezone(ist_timezone)
    time = dict()
    time['day'] = dt_ist.strftime('%A')
    time['month'] = dt_ist.strftime('%B')
    time['date'] = dt_ist.strftime('%Y-%m-%d')
    time['time'] = dt_ist.strftime('%H:%M')
    return time

In [6]:
llm_with_tools = llm.bind_tools([get_user_data, current_date_time])

tool_mapping = {
    "get_user_data" : get_user_data,
    "current_date_time" : current_date_time
}

In [7]:
#Refresh convo.

template = """You are an intelligent virtual financial agent helping our customer {first_name} {last_name}.
            Your role is to help manage the customer's loan repayment and answer their financial questions in a clear and precise way. 

            Instructions:
            1. Use precise financial language and ensure clear, accurate information.
            2. If the user is willing to pay the loan then please provide this link '''https://paymentUSER1UDN.com'''. Do not send the link until user requests or user wants to pay the loan.
            3. If the customer is struggling, provide options like grace periods, payment restructuring, or deadline extensions considering their income, number of late repayment and loan amount yet to be repayed.
            4. Keep responses short and to the point.
            5. Ensure confidentiality and remind the customer to keep their payment details secure.
            6. You can only extend the last loan repayment date by a maximum of 10 days if user requests for grace periods or deadline extensions considering their income, number of late repayment and loan amount yet to be repayed.
            7. If the question cannot be answered using the information provided, reply with "Sorry, but I am unable to answer this query". 
            
"""

chat_history = []

In [8]:
template_messages = [
    SystemMessage(content=template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{query}")
    #HumanMessage(content="{query}")
]

prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [9]:
phone = 9804604602

In [10]:
user_data = get_info(phone)

user_info = {
    "first_name": user_data['first_name'],
    "last_name": user_data['last_name']
}

In [11]:
formatted_template = template.format(**user_info)
template_messages[0] = SystemMessage(content=formatted_template)
prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [12]:
def chat(text:str) -> str:
    messages = prompt_template.format_messages(
        chat_history=chat_history,
        query = text
    )
    chat_history.append(HumanMessage(text))
    
    response = llm_with_tools.invoke(messages)
    chat_history.append(response)
    
    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool = tool_mapping[tool_call['name']]
            output = tool.invoke(tool_call['args'])
            chat_history.append(ToolMessage(str(output), tool_call_id=tool_call['id']))
            ai_says = llm_with_tools.invoke(chat_history)
        chat_history.append(ai_says)
    else:
        ai_says = response

    return ai_says.content

In [13]:
while True:
    query = input('You Said:')
    if query is not None:
        if query.lower() == 'thanks':
            break
        else:
            res = chat(query)
            print('Bot: ',res)

You Said: Hello


Bot:  Hello Diya! I'm here to help with your loan management needs. How can I assist you today?


You Said: What is my phone number?


Bot:  Sorry, but I am unable to answer this query.


You Said: What is my loan amount?


Bot:  Your loan amount is 36100.6 INR.


You Said: thanks


In [15]:
chat_history

[HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Diya! I'm here to help with your loan management needs. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 372, 'total_tokens': 395, 'completion_tokens': 23}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-65dfc51d-3820-47ae-bd63-23e3b0b46984-0', usage_metadata={'input_tokens': 372, 'output_tokens': 23, 'total_tokens': 395}),
 HumanMessage(content='What is my phone number?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Sorry, but I am unable to answer this query.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 404, 'total_tokens': 415, 'completion_tokens': 11}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-31f20744-bb62-43e4-b608-64ad27c6dbf4-0', usage_metadata={'input_tokens': 404, 'output_tokens': 11, 'total_tokens': 415}),
 HumanMessage(conten